In [1]:
# from tqdm.contrib.concurrent import process_map
# import pathlib
# from safetensors.torch import save_file
# 
# import torch
# import torchaudio
# import os
# 
# def get_output_path(path: pathlib.Path) -> str:
#     return str(path).replace("songs", "tensor_songs").replace("mp3", "safetensors")
# 
# def convert_audio_to_tensor(path: pathlib) -> None:
#     output_path: pathlib.Path = pathlib.Path(get_output_path(path))
#     output_path.parent.mkdir(parents=True, exist_ok=True)
#     if output_path.exists():
#         return
#     
#     audio: torch.Tensor
#     audio, sample_rate = torchaudio.load(path)
#     save_file(
#         {"audio": audio.contiguous(), "sample_rate":torch.tensor(sample_rate, dtype=torch.int32)},
#         output_path
#     )
# 
# songs_path: list[pathlib.Path] = list((
#         pathlib.Path(pathlib.Path.cwd()).parent / "data" / "raw" / "songs"
# ).rglob("*.mp3"))
# 
# r = process_map(convert_audio_to_tensor, songs_path, max_workers=os.cpu_count(), chunksize=1)

In [2]:
!du -sh /home/paolo/git/spotify-playlist-generator/data/raw/songs

11G	/home/paolo/git/spotify-playlist-generator/data/raw/songs


In [3]:
!du -sh /home/paolo/git/spotify-playlist-generator/data/raw/tensor_songs

172G	/home/paolo/git/spotify-playlist-generator/data/raw/tensor_songs


## Convert audio to spectrogram image and save it

In [18]:
from tqdm.contrib.concurrent import process_map
import pathlib

import torch
import torchaudio
import torchaudio.transforms as T
import os
import torchvision
import torchvision.transforms.v2 as v2

SOURCE_FOLDER: str = "songs"
SOURCE_FORMAT: str = ".mp3"
OUTPUT_FOLDER: str = "images"
OUTPUT_FORMAT: str = "png"

class AddChannel(torch.nn.Module):
    def __init__(self, channel_type: str = "mean"):
        assert channel_type in {"mean", "sum", "zero"}
        super().__init__()
        self.channel_type = channel_type
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.channel_type == "mean":
            channel: torch.Tensor = x.mean(dim=0).unsqueeze(dim=0)
        elif self.channel_type == "sum":
            channel: torch.Tensor = x.sum(dim=0).unsqueeze(dim=0)
        else:
            channel: torch.Tensor = torch.zeros((1, x.shape[1], x.shape[2]))
        
        return torch.cat([x, channel], dim=0)


def get_output_path(path: pathlib.Path) -> str:
    return str(path).replace(SOURCE_FOLDER, OUTPUT_FOLDER).replace(SOURCE_FORMAT, OUTPUT_FORMAT)

def convert_audio_to_tensor(path: pathlib) -> None:
    output_path: pathlib.Path = pathlib.Path(get_output_path(path))
    output_path.parent.mkdir(parents=True, exist_ok=True)
    if output_path.exists():
        return

    audio: torch.Tensor
    audio, sample_rate = torchaudio.load(path)
    
    return v2.Compose([
        T.Spectrogram(
            n_fft=1024,
            win_length=1024,
            hop_length=256
        ),
        AddChannel(),
        # v2.ToDtype(torch.float32, scale=True)
    ])(audio)

songs_path: list[pathlib.Path] = list((pathlib.Path(pathlib.Path.cwd()).parent / "data" / "raw" / SOURCE_FOLDER).rglob(f"*{SOURCE_FORMAT}"))

audio = convert_audio_to_tensor(songs_path[0])
audio.shape, audio.min(), audio.max()
# r = process_map(convert_audio_to_tensor, songs_path, max_workers=os.cpu_count(), chunksize=1)

(torch.Size([3, 513, 52295]), tensor(0.), tensor(45887.6406))

In [16]:
torchvision.io.write_png(audio, filename=get_output_path(audio))

RuntimeError: Input tensor dtype should be uint8